추가중

## 움직이는 그래프
우리가 지금까지 그린 그래프는 사진처럼 멈춰있었다.

sine 함수나 cosine함수가 진행하는걸 보고 싶다면? 파동함수가 진행하는 것을 보고 싶다면???

다행히도 MatPlotLib에는 그래프가 움직이도록 만들어주는 라이브러리도 있다.

## 3.3.0 FFmpeg 설치
움직이는 그래프는 동영상을 만드는것이므로 동영상을 만들어줄 FFmgeg라는 인코더가 필요하다.

설치법은 아래 사이트에서 확인하자.

https://www.wikihow.com/Install-FFmpeg-on-Windows

## 3.3.1 움직이는 동영상 만들기

FFmpeg 설치를 완료했다면, sin함수가 진행하는 그래프를 동영상으로 만들어 보자

1.Matplotlib Animation Tutorial

https://matplotlib.org/examples/animation/index.html

https://jakevdp.github.io/blog/2012/08/18/matplotlib-animation-tutorial/
